# Load data

In [1]:
import numpy as np 
from monty.json import MontyDecoder
from monty.serialization import loadfn

data = loadfn('bulk_moduli.json')
structures = data['structures']
targets = np.log10(data['bulk_moduli'])

# Set up model and train

In [2]:
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.models import MEGNetModel

model = MEGNetModel(10, 2, nblocks=1, lr=1e-2,
                    n1=4, n2=4, n3=4, npass=1, ntarget=1,
                    graph_converter=CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 10), 0.5)))

C:\Users\gnypj\Projects\megnet\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
model.train(structures, targets, epochs=2)

Epoch 1/2


C:\Users\gnypj\Projects\megnet\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 3.9623
Epoch 2/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step - loss: 3.7588


# Prediction

## Predict from structure

In [6]:
from pymatgen.ext.matproj import MPRester
mpr = MPRester(api_key='YQeZsht7e8JJsB2EMnkQq146bWfWBDaj')

# let's get some structure
structure = mpr.get_structure_by_material_id('mp-1096926')

In [7]:
predicted_K = 10 ** model.predict_structure(structure).ravel()
print('The predicted K for {} is {} GPa'.format(structure.formula, predicted_K[0]))

The predicted K for Cs2 In1 Ag1 Cl6 is 1.2769129276275635 GPa
